# Lexer

Today, we will learn all about the **lexing phase** of a compiler.

For example,

In [1]:
let source_code : string =
"x := (3 + 4) + 5
read y
x + y"

val source_code : string = "x := (3 + 4) + 5\nread y\nx + y"


is transformed in the list of tokens: 

```let phrasal_syntax : token list =
[ID "x"; ASGN; LBRA; INT 3; PLUS; INT 4; RBRA; PLUS; INT 5; READ;
 ID "y"; ID "x"; PLUS; ID "y"]```

## Manual Lexers

We start with expressions and their representation in abstract syntax: 

In [2]:
type op = Plus | Minus | Mult | Div
type exp = Id of string | Numb of int | Op of exp * op * exp 

type op = Plus | Minus | Mult | Div


type exp = Id of string | Numb of int | Op of exp * op * exp


We start off with a lexer for expressions without identifers or integers, the only constant is ``1`` - represented by the token ``ONE``. 

We represent tokens as an **algebraic datatype**.
If you require a quick reminder you can find some good explanation here: https://cs3110.github.io/textbook/chapters/data/variants.html and here: https://cs3110.github.io/textbook/chapters/data/algebraic_data_types.html. 

In [3]:
 type token = ONE
           | PLUS  | MINUS | STAR | SLASH 
           | LBRA | RBRA 
           | EOF

type token = ONE | PLUS | MINUS | STAR | SLASH | LBRA | RBRA | EOF


Recall that we convert a String into a list of tokens.
For this representation we want to look through all characters one-by-one.

Below you see a function ``lex_step`` which takes a list of characters and yields the next token and the remaining list of characters. 

If the list of characters ``cs``... 
- is empty, we return the end-of-file symbol and the empty list. 
- is a white space character, we forget this character and go on with the remaining list of characters. 
- is '1', we return the token ``ONE`` and the remaining list of characters. 
- ... similarly for our other defined tokens. 
- If it is none of these tokens, we raise an exception.

In [4]:
exception LexError

let rec lex_step (cs: char list) : token * char list = 
  match cs with 
  | [] -> (EOF, [])
  | '\t' :: cs | '\n' :: cs | ' ' :: cs -> lex_step cs
  | '1' :: cs -> (ONE, cs)
  | '+' :: cs -> (PLUS, cs)
  | '-' :: cs -> (MINUS, cs)
  | '*' :: cs -> (STAR, cs) 
  | '/' :: cs -> (SLASH, cs)  
  | '(' :: cs -> (LBRA, cs) 
  | ')' :: cs -> (RBRA, cs)
  | c :: cs -> raise LexError

exception LexError


val lex_step : char list -> token * char list = <fun>


Internally strings are not lists of characaters. 
We hence have convert strings to a list of letters (``explode``) and convert a list of letters to a string (``implode``). 
Don't worry about how they are defined!

In [5]:
let implode chars = 
  let buf = Buffer.create 16 in
  List.iter (Buffer.add_char buf) chars;
  Buffer.contents buf

let explode s =
  let rec exp i l =
    if i < 0 then l else exp (i - 1) (s.[i] :: l) in
  exp (String.length s - 1) []  

val implode : char list -> string = <fun>


val explode : string -> char list = <fun>


We can then test our above function ``lex_step``: 

In [6]:
lex_step (explode "1  + 1")

- : token * char list = (ONE, [' '; ' '; '+'; ' '; '1'])


In [7]:
lex_step (explode " (1  + 1")

- : token * char list = (LBRA, ['1'; ' '; ' '; '+'; ' '; '1'])


To define the full lexing phase we repeat reading the next token, until we get to the end of the file: 

In [8]:
let rec lex cs = 
    match lex_step cs with 
    | (EOF, _) -> []
    | (t, cs) -> t :: lex cs

val lex : char list -> token list = <fun>


In [9]:
lex (explode "1  + 1")

- : token list = [ONE; PLUS; ONE]


In [10]:
lex (explode " (1  + 1")

- : token list = [LBRA; ONE; PLUS; ONE]


## Optional: A Manual Lexer for Expressions

(The following is optional and was not handled in the lecture.)

Let's now go to full expressions and the following token type:

In [11]:
type token = ID of string | INT of int
           | PLUS  | MINUS | STAR | SLASH 
           | LBRA | RBRA 
           | EOF

type token =
    ID of string
  | INT of int
  | PLUS
  | MINUS
  | STAR
  | SLASH
  | LBRA
  | RBRA
  | EOF


Implementing a lexer which recognizes identifers and integers requires us to implement the maximal munch rule into our lexer function. 

The following function ``grab`` continues reading an input, until the predicate ``f`` no longer holds:

In [12]:
let rec grab (f : char -> bool) (xs : char list) : char list * char list  = match xs with 
| [] -> ([], [])
| x :: xs' -> let (l1, l2) = grab f xs' in
                if f x then (x :: l1, l2) else ([], xs)

val grab : (char -> bool) -> char list -> char list * char list = <fun>


We can moreover define functions which recognize whether a character is a digit or a character by a case distinction:

In [13]:
let is_digit digit =
  match digit with
    '0' .. '9' -> true
  | _ -> false

let is_char char = 
  match char with 
    'a' .. 'z' -> true 
  | 'A' .. 'Z' -> true
  | _ -> false

val is_digit : char -> bool = <fun>


val is_char : char -> bool = <fun>


This allows us to define functions which read an integer or an identifier:

In [14]:
let lex_int s = let (l1, l2) = grab is_digit s in 
                (INT (int_of_string (implode l1)), l2)

let lex_id s = let (l1, l2) = grab is_char s in 
                (ID (implode l1), l2)

val lex_int : char list -> token * char list = <fun>


val lex_id : char list -> token * char list = <fun>


... and hence also to define a lexer function as before:

In [15]:
let rec lex_step s : token * char list = 
  match s with 
  | [] -> (EOF, [])
  | '\t' :: xs | '\n' :: xs | ' ' :: xs -> lex_step xs
  | '+' :: xs -> (PLUS, xs)
  | '-' :: xs -> (MINUS, xs)
  | '*' :: xs -> (STAR, xs) 
  | '/' :: xs -> (SLASH, xs)  
  | '(' :: xs -> (LBRA, xs) 
  | ')' :: xs -> (RBRA, xs)
  | c :: cs -> if is_char c 
                  then lex_id (c :: cs)
                  else if is_digit c then lex_int (c :: cs)
                  else raise LexError

val lex_step : char list -> token * char list = <fun>


In [16]:
let rec lex cs = 
    match lex_step cs with 
    | (EOF, _) -> []
    | (t, cs) -> t :: lex cs

val lex : char list -> token list = <fun>


Let's try it out:

In [17]:
lex (explode "34 + abc")

- : token list = [INT 34; PLUS; ID "abc"]


## Lexer Generator for Expressions

We show how to use the lexer generator in the Jupyter notebook.

We will here use the ``ocamllex`` tool - this one is based on the ``lex`` tool. 

We require certain functionality from Jupyter notebooks to later run ``ocamllex`` and compile files.

In [18]:
#require "jupyter.notebook" ;;

open Jupyter_notebook;;

/home/opam/.opam/4.13/lib/base64: added to search path
/home/opam/.opam/4.13/lib/base64/base64.cma: loaded
/home/opam/.opam/4.13/lib/ocaml/compiler-libs: added to search path
/home/opam/.opam/4.13/lib/ocaml/compiler-libs/ocamlcommon.cma: loaded
/home/opam/.opam/4.13/lib/seq: added to search path
/home/opam/.opam/4.13/lib/yojson: added to search path
/home/opam/.opam/4.13/lib/yojson/yojson.cma: loaded
/home/opam/.opam/4.13/lib/ppx_yojson_conv_lib: added to search path
/home/opam/.opam/4.13/lib/ppx_yojson_conv_lib/ppx_yojson_conv_lib.cma: loaded
/home/opam/.opam/4.13/lib/ocaml/unix.cma: loaded
/home/opam/.opam/4.13/lib/bytes: added to search path
/home/opam/.opam/4.13/lib/uuidm: added to search path
/home/opam/.opam/4.13/lib/uuidm/uuidm.cma: loaded
/home/opam/.opam/4.13/lib/jupyter: added to search path
/home/opam/.opam/4.13/lib/jupyter/jupyter.cma: loaded
/home/opam/.opam/4.13/lib/result: added to search path
/home/opam/.opam/4.13/lib/result/result.cma: loaded
/home/opam/.opam/4.13/lib/

This is the content of the file ``exp.mll``, the input to the later lexer generator. 
See the slides for the explanation.

```
{ (* User definitions *) 

type token = ID of string | INT of int
           | PLUS  | MINUS | STAR | SLASH 
           | LBRA | RBRA | EOF
           
exception LEX of string

}

(* LEX definitions - setting up regular expressions for int/white/id *) 
let int = '-'? ['0'-'9'] ['0'-'9']*
let white = [' ' '\t' '\n']+
let id = ['a'-'z' 'A'-'Z' '_'] ['a'-'z' 'A'-'Z' '0'-'9' '_']*

(* Rules *) 
rule token = parse
    | white          { token lexbuf }
    | int            { INT (int_of_string (Lexing.lexeme lexbuf)) }
    | id             { ID (Lexing.lexeme lexbuf) }
    | '+'            { PLUS }
    | '-'            { MINUS }
    | '*'            { STAR }
    | '/'            { SLASH }
    | '('            { LBRA }
    | ')'            { RBRA }
    | eof            { EOF }
    | _ { raise (LEX ("Unexpected char: " ^ Lexing.lexeme lexbuf)) } 
    
    
(* User functions *)   
```

The first command calls the lexer generator on the input file ``exp.mll`` and saves the output to ``exp.ml``.

The second command compiles the output file into ``exp.cmi``, a compiled interface file which includes e.g. type information of functions or variables and ``exp.cmo``, a bytecode object file (think of .class files in Java).

(You don't need to know about these processes, we will always give you the corresponding code.)

In [19]:
Process.sh "ocamllex exp.mll";;
Process.sh "ocamlc -c exp.ml";;

12 states, 417 transitions, table size 1740 bytes


- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0; stdout = None;
 stderr = None}


- : Jupyter_notebook.Process.t =
{Jupyter_notebook.Process.exit_status = Unix.WEXITED 0; stdout = None;
 stderr = None}


You can see the result in the file ``exp.ml``.
(It is strongly recommend to look yourself!)

The ``#load`` command allows to include the functions in the compiled file.

In [20]:
#load "exp.cmo"

All functions we use from this file will have to be prefixed by ``Exp.`` (the name of the file capitalized). 

E.g., we can check the type of the function ``token``: 

In [21]:
Exp.token

- : Lexing.lexbuf -> Exp.token = <fun>


So the function ``Exp.token`` expects a buffer (something of type ``Lexing.lexbuf``) and then returns the next token. 

The library contains a function ``Lexing.from_string`` which converts a string into a buffer. 

In principle, we can now use the function ``Exp.token`` to get the next element of the buffer:

In [22]:
let buf = Lexing.from_string "3 + 5";; 

Exp.token buf;;

Exp.token buf;;

Exp.token buf;;

Exp.token buf

val buf : Lexing.lexbuf =
  {Lexing.refill_buff = <fun>; lex_buffer = Bytes.of_string "3 + 5";
   lex_buffer_len = 5; lex_abs_pos = 0; lex_start_pos = 0; lex_curr_pos = 0;
   lex_last_pos = 0; lex_last_action = 0; lex_eof_reached = true;
   lex_mem = [||];
   lex_start_p =
    {Lexing.pos_fname = ""; pos_lnum = 1; pos_bol = 0; pos_cnum = 0};
   lex_curr_p =
    {Lexing.pos_fname = ""; pos_lnum = 1; pos_bol = 0; pos_cnum = 0}}


- : Exp.token = Exp.INT 3


- : Exp.token = Exp.PLUS


- : Exp.token = Exp.INT 5


- : Exp.token = Exp.EOF


We could use the definition of a buffer but to make things clearer, we will work with lists. 

We can convert the stream of tokens into lists by the following function ``stream_to_list``, which takes one-by-one the next element of the buffer (``Exp.token buffer``) and makes a case analysis whether 
- this element is the end-of-file symbol ``EOF`` - we can return the empty list. 
- this element is another token - then remember this token t, and put it to the beginning of the list and run the function recursively on the remaining buffer.

In [23]:
let rec stream_to_list buffer = 
    match Exp.token buffer with 
    | EOF -> []
    | t -> t :: stream_to_list buffer

val stream_to_list : Lexing.lexbuf -> Exp.token list = <fun>


See below for an example of getting the corresponding list:

In [24]:
let res = stream_to_list (Lexing.from_string "3 + 5")

val res : Exp.token list = [Exp.INT 3; Exp.PLUS; Exp.INT 5]


Try it out with your own example: 

In [25]:
(* TODO *)